# vLLM

推荐您尝试使用 [vLLM](https://github.com/vllm-project/vllm) 来部署 Qwen 模型。它易于使用，且具备极高的服务吞吐量。其核心特性包括 PagedAttention（高效管理注意力键值内存）、连续批处理（continuous batching）以及优化的 CUDA 内核等。

欲深入了解 vLLM，请参阅[论文](https://arxiv.org/abs/2309.06180)和[官方文档](https://docs.vllm.ai/)。

**部署环境说明**：
- 平台：Google Colab
- 操作系统：Ubuntu 22.04
- GPU：Tesla T4 (15GB 显存)
- 目标受众：智能体中高级学习者，关注高性能推理部署

## 环境设置 (Environment Setup)

默认情况下，您可以在纯净环境中使用 pip 安装 `vllm`：

In [ ]:
!pip install "vllm>=0.8.5"

**注意**：`vllm` 对 `torch` 及其 CUDA 版本有严格的依赖要求。如果遇到问题，请查阅[官方安装文档](https://docs.vllm.ai/en/latest/getting_started/installation.html)。

## API 服务 (API Service)

使用 vLLM 构建兼容 OpenAI 的 API 服务非常便捷。默认端口为 `http://localhost:8000`。

针对 Colab 的 T4 GPU 环境，我们将部署 **AWQ 量化版本** 的 `Qwen/Qwen3-8B-AWQ` 模型。

In [ ]:
import time
import requests
import os

# 启动 vLLM 服务器（后台运行）
# 适配 T4 环境的关键参数：
# --quantization awq: 显式指定使用 AWQ 量化后端
# --dtype float16: 配合量化使用
# --max-model-len 32768: 根据显存情况可能需要调整，这里保持默认尝试
print("正在启动 vLLM 服务器... (预计需要几分钟下载和加载模型)")

# 注意：vLLM 可能会占用大量显存，若遇到 OOM，请尝试减小 max-model-len，例如 --max-model-len 8192
get_ipython().system_raw('nohup vllm serve Qwen/Qwen3-8B-AWQ --port 8000 --quantization awq --dtype float16 > vllm.log 2>&1 &')

# 等待服务器就绪
def wait_for_server(url, timeout=900):
    start_time = time.time()
    print("等待服务就绪...", end="")
    while time.time() - start_time < timeout:
        try:
            response = requests.get(f"{url}/health")
            if response.status_code == 200:
                print("\n服务器已启动并就绪！")
                return True
        except requests.ConnectionError:
            pass
        time.sleep(10)
        print(".", end="", flush=True)
    print("\n服务器启动超时。请运行 `!cat vllm.log` 查看错误日志。")
    return False

wait_for_server("http://127.0.0.1:8000")

### 基本用法 (Basic Usage)

使用 `curl` 与 Qwen 进行交互：

In [ ]:
!curl http://localhost:8000/v1/chat/completions \
  -H "Content-Type: application/json" \
  -d '{
  "model": "Qwen/Qwen3-8B-AWQ",
  "messages": [
    {"role": "user", "content": "请简要介绍一下大型语言模型。"}
  ],
  "temperature": 0.6,
  "top_p": 0.95,
  "top_k": 20,
  "max_tokens": 1024
}'

### 思考模式与非思考模式

与 SGLang 类似，vLLM 也支持通过 `chat_template_kwargs` 控制 Qwen3 的思考行为。以下是禁用思考的示例：

In [ ]:
!curl http://localhost:8000/v1/chat/completions \
  -H "Content-Type: application/json" \
  -d '{
  "model": "Qwen/Qwen3-8B-AWQ",
  "messages": [
    {"role": "user", "content": "请简要介绍一下大型语言模型。"}
  ],
  "temperature": 0.7,
  "top_p": 0.8,
  "top_k": 20,
  "max_tokens": 1024,
  "presence_penalty": 1.5,
  "chat_template_kwargs": {"enable_thinking": false}
}'

### 解析思考内容 (Parsing Thinking Content)

vLLM 支持解析思考内容。
- 启动参数：`--enable-reasoning --reasoning-parser deepseek_r1` (或 v0.9.0+ 的 `qwen3`)
- 注意：vLLM 0.8.5 中，`enable_thinking=False` 可能不兼容此功能。建议在 v0.9.0+ 版本中使用。

### 常见问题 (FAQ)：OOM (显存溢出)

在 Colab 等显存有限的环境中，OOM 是常见问题。建议尝试以下参数进行优化：
1. `--max-model-len`: 减小上下文长度。例如设置为 `8192` 或更小，以减少显存占用。
2. `--gpu-memory-utilization`: vLLM 默认占用 90% 显存。如果遇到 OOM，可以尝试微调此参数。


## Python 库使用 (Python Library)

vLLM 也可以作为 Python 库直接使用，方便离线批处理。

**重要提示**：在 Colab 单卡 T4 环境下，同时运行 vLLM 服务器和 Python 库实例会导致 **OOM**。在大约运行下面的代码之前，**必须**先停止上面的后台服务器。

In [ ]:
# 首先停止后台的 vLLM 服务器
!pkill -f vllm
print("vLLM 服务器已停止。现在可以运行 Python 库示例。")
# 稍微等待显存释放
time.sleep(5)

In [ ]:
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams
import torch

# 清理缓存以确保有足够显存
torch.cuda.empty_cache()

# 初始化分词器
try:
    tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-8B-AWQ")

    # 配置采样参数
    # 请根据显存调整 max_tokens
    sampling_params = SamplingParams(temperature=0.6, top_p=0.95, top_k=20, max_tokens=1024)

    # 初始化 vLLM 引擎
    # 显式指定 quantization='awq' 以加载 AWQ 模型
    llm = LLM(model="Qwen/Qwen3-8B-AWQ", quantization="awq", dtype="float16", max_model_len=8192)

    # 准备输入
    prompt = "请简要介绍一下大型语言模型。"
    messages = [
        {"role": "user", "content": prompt}
    ]
    
    # 应用聊天模板
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        # enable_thinking=True, # 如果模型支持且库版本支持
    )

    # 生成输出
    outputs = llm.generate([text], sampling_params)

    # 打印输出
    for output in outputs:
        prompt = output.prompt
        generated_text = output.outputs[0].text
        print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")
        
except Exception as e:
    print(f"运行出错 (可能是 OOM): {e}")